**I import some useful libraries in python.**

In [ ]:
import json
import csv
!pip install unidecode
from unidecode import unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 2.2 MB/s eta 0:00:00


**I mount google drive because all the files (for example .csv) that i used for the implementation of my thesis are saved on my google drive.**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Reading the test dataset that includes answerable and non-answerable questions over wikidata and the corresponding test dataset that includes only answerable questions over wikidata that are in .txt file format and droping of the column of their answers ids.**

In [ ]:
import pandas as pd
test_dataset = pd.read_csv('annotated_wd_data_test.txt', sep="\t", names=['entity_id','relation_id', 'answer', 'Question']) #read test dataset
test_answerable_dataset = pd.read_csv('annotated_wd_data_test_answerable.txt', sep="\t", names=['entity_id','relation_id', 'answer', 'Question']) #read test dataset
test_dataset = test_dataset.drop(labels='answer', axis=1) #drop answer
test_answerable_dataset = test_answerable_dataset.drop(labels='answer', axis=1) #drop answer

**I made some preprocessing for each question by creating the function preprocessing, which takes as argument the text (in this case the question) and performs accent removal, as well as punctuation mark removal (period, question mark, exclamation mark), s removal with apostrophe('s) and comma(,) removal.Finally this function, which is shown below, returns the original text with the above mentioned processing.**

In [ ]:
def Preprocessing(text):
    text = unidecode(text.replace("?", "").replace("'s", "").replace(".", "").replace("!", "").replace(",", ""))
    return text

test_dataset['Question'] = test_dataset['Question'].apply(lambda x:Preprocessing(x))
test_answerable_dataset['Question'] = test_answerable_dataset['Question'].apply(lambda x:Preprocessing(x))

**The SPARQLWrapper library is downloaded, which contributes to the generation of SPARQL queries, in order to find the entity labels.**

In [ ]:
#agent='WDQS-example Python'
#Chicobot-Agent
!pip install sparqlwrapper
import sys
from SPARQLWrapper import SPARQLWrapper, JSON, CSV
sparql = SPARQLWrapper("https://query.wikidata.org/sparql", agent= "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1]))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 5.2 MB/s eta 0:00:00


**The function find\_divisors which receives as an argument the length of either the SimpleQuestions training or the SimpleQuestions validation dataset and returns the divisors of this number, has as aim to find the divisors of the lengths of of the full SimpleQuestions test dataset, so that it is then possible to get the entity labels from SPARQL as quickly as possible.**

In [ ]:
def find_divisors(a):
    divisors = []
    for i in range(1, a + 1):
        if a % i == 0:
            divisors.append(i)
    return divisors

# Example usage
number_to_check_1 = len(test_dataset)
number_to_check_2 = len(test_answerable_dataset)
result_1 = find_divisors(number_to_check_1)
result_2 = find_divisors(number_to_check_2)
print(f"The divisors of {number_to_check_1} are: {result_1}")
print(f"The divisors of {number_to_check_2} are: {result_2}")

The divisors of 9961 are: [1, 7, 1423, 9961]
The divisors of 5622 are: [1, 2, 3, 6, 937, 1874, 2811, 5622]


**The SPARQL Queries below found from wikidata the entity labels and the entity ids for each entity id (of course for each question) of the SimpleQuestions for both test dataset that includes answerable and non-answerable questions over wikidata and for the test dataset that includes only answerable questions over wikidata.**

In [ ]:
test_entity_names = []
test_entity_ids = []
for i in range(0,len(test_dataset),7):  #retrieve training set entities' corresponding ids
  str = ""
  for entity_id in test_dataset['entity_id'][i:i+7]:
   str = str + "wd:" + entity_id + " "
  sparql.setQuery("""
  SELECT ?item ?itemLabel
  WHERE
  {
    VALUES ?item {""" + str + """}
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
  }
""")
  sparql.setReturnFormat(JSON)
  results = sparql.query().convert()
  test_results_df = pd.json_normalize(results['results']['bindings'])
  for j in test_results_df['item.value']:
      test_entity_ids.append(j)
  for j in test_results_df['itemLabel.value']:
      test_entity_names.append(j)

In [ ]:
test_answerable_entity_names = []
test_answerable_entity_ids = []
for i in range(0,len(test_answerable_dataset),3):  #retrieve training set entities' corresponding ids
  str = ""
  for entity_id in test_answerable_dataset['entity_id'][i:i+3]:
   str = str + "wd:" + entity_id + " "
  sparql.setQuery("""
  SELECT ?item ?itemLabel
  WHERE
  {
    VALUES ?item {""" + str + """}
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
  }
""")
  sparql.setReturnFormat(JSON)
  results = sparql.query().convert()
  test_answerable_results_df = pd.json_normalize(results['results']['bindings'])
  for j in test_answerable_results_df['item.value']:
      test_answerable_entity_ids.append(j)
  for j in test_answerable_results_df['itemLabel.value']:
      test_answerable_entity_names.append(j)

**Creation of the corresponding lists containing the questions, the entity labels, the relation ids and the entity ids of the initial SimpleQuestions test dataset, either this dataset includes answerable and answerable questions over wikidata questions or only answerable questions over wikidata.**

In [ ]:
test_dataset_questions = test_dataset['Question'].to_list()
test_dataset_entity_ids = test_dataset['entity_id'].to_list()
test_dataset_answerable_questions = test_answerable_dataset['Question'].to_list()
test_dataset_answerable_entity_ids = test_answerable_dataset['entity_id'].to_list()

**The entity ids that are occurred from SPARQL are expressions that contain an encoded name, for example for the question of the SimpleQuestions (full) training dataset "who is a musician born in detroit", the entity id that occurred is https://www.wikidata.org/wiki/Q12439, which is related with Detroit (Q12439, if i search in wikidata).**

**So the python function entity\_ids is created with the aim to split the full expression with the entity id, such as the example above and to keep the encoded name, i.e named start from "Q" ("Q12439" in the above example).**

In [ ]:
#Split Spaqrl URI's in order to get the entity ids
import os
def enitity_ids(ids): #get ids from sparql query ids
 entity_ids = []
 for i in range(len(ids)):
   entity_ids.append(ids[i].split(os.path.sep)[-1])
 return entity_ids
test_ids = enitity_ids(test_entity_ids)
test_answerable_ids = enitity_ids(test_answerable_entity_ids)
print(len(test_ids),test_ids)
print(len(test_answerable_ids),test_answerable_ids)
print(len(test_dataset['entity_id']))
print(len(test_answerable_dataset['entity_id']))

9969 ['Q5487302', 'Q16330302', 'Q16225521', 'Q7358590', 'Q154335', 'Q1761', 'Q11272426', 'Q2581168', 'Q2747238', 'Q62498', 'Q182485', 'Q12152', 'Q1176417', 'Q7123909', 'Q1189775', 'Q276817', 'Q472382', 'Q7443093', 'Q7559501', 'Q615196', 'Q1456475', 'Q21590660', 'Q1568', 'Q79952', 'Q4763200', 'Q439893', 'Q5095381', 'Q64412', 'Q1753186', 'Q607806', 'Q1351994', 'Q913582', 'Q1304968', 'Q270', 'Q1902406', 'Q183504', 'Q3630', 'Q1085', 'Q8063973', 'Q1513688', 'Q2629576', 'Q4786243', 'Q4979845', 'Q11399', 'Q23622', 'Q316231', 'Q17036469', 'Q7508530', 'Q3533158', 'Q1386899', 'Q7749', 'Q137259', 'Q3567556', 'Q318044', 'Q3007916', 'Q1054574', 'Q544899', 'Q6134495', 'Q6650386', 'Q2086913', 'Q6885337', 'Q6465668', 'Q3534902', 'Q7579116', 'Q2662597', 'Q3701781', 'Q3790674', 'Q1114984', 'Q6918683', 'Q1801175', 'Q56091', 'Q170238', 'Q15461094', 'Q2325140', 'Q280092', 'Q6959936', 'Q27306', 'Q202585', 'Q16249575', 'Q37073', 'Q4959914', 'Q1627134', 'Q7283066', 'Q6505921', 'Q312480', 'Q2724377', 'Q464605'

**The function labels from which i confirm the entity id of each question, which occurred from SPARQL, through the entity id obtained for each question from the transfer of SimpleQuestions from Freebase to Wikidata. This function takes as arguments the list of questions, the list of the entity ids both from the corresponding SimpleQuestions dataset column and from the entity\_ids function and the list of entity labels occured from SPARQL.**

**Essentially i assigned to each question and entity id its corresponding entity label by searching in the list of test entity ids obtained from SPARQL. If any of these entity ids exist in the SimpleQuestions training and validation entity labels lists, that are created before and matching the entity label and thus a new column with entity labels for the training and the validation dataset.**

In [ ]:
import operator as op
def labels(questions, entity_id_dataset, entity_ids, entity_names):# entity_ids and dataset must have the same length
#In this step we find the entity ids from the initial dataset to the list occured from Sparql Query
#for each of the train and validation dataset.
#This will become the train or validation data new column, containing entities' labels

  label_df_column = []
  for entity_id in entity_id_dataset:
   j=0
   for ent in entity_ids: #find its id on list
    if entity_id == ent:
      label_df_column.append(entity_names[j]) #and find corresponding label
      break
    j += 1
  return label_df_column


test_dataset['entity_label'] = labels(test_dataset_questions, test_dataset_entity_ids, test_ids, test_entity_names)
print(test_dataset['entity_label'], len(test_dataset['entity_label']))
test_answerable_dataset['entity_label'] = labels(test_dataset_answerable_questions, test_dataset_answerable_entity_ids, test_answerable_ids, test_answerable_entity_names)
print(test_answerable_dataset['entity_label'], len(test_answerable_dataset['entity_label']))

0            Harder ... Faster
1                  Alex Golfis
2                     Phil Hay
3                Roger Marquis
4                   Yves Klein
                 ...          
9956            Doctor Faustus
9957             Oklahoma City
9958               2974 Holden
9959    Snow Falling on Cedars
9960           Lucille Clifton
Name: entity_label, Length: 9961, dtype: object 9961
0           Roger Marquis
1              Yves Klein
2            Carlos Gómez
3       Engelbert Zaschka
4           Pee Wee Reese
              ...        
5617            Barcelona
5618             Jun Lana
5619      Gunnar Johansen
5620          2974 Holden
5621      Lucille Clifton
Name: entity_label, Length: 5622, dtype: object 5622


**The preprocessing of the entity labels column occurred from labels function is made through function Preprocessing that is created previously. The reindexing of the columns (entity id, relation id, questions, entity label) of the initial SimpleQuestions dataset (either full or the dataset containing only questions that are answerable over wikidata),in order to include the entity labels as a new column in the initial SimpleQuestions dataset, is achieved by using the reindex() method from pandas.**

In [ ]:
test_dataset['entity_label'] = test_dataset['entity_label'].apply(lambda x:Preprocessing(x))
new_columns = ["entity_id","entity_label","relation_id","Question"]
test_dataset = test_dataset.reindex(columns=new_columns)
print(test_dataset)

      entity_id            entity_label relation_id  \
0      Q5487302          Harder  Faster        P136   
1     Q16330302             Alex Golfis         P19   
2     Q16225521                Phil Hay         R58   
3      Q7358590           Roger Marquis         P20   
4       Q154335              Yves Klein        P509   
...         ...                     ...         ...   
9956   Q1447249          Doctor Faustus        P170   
9957     Q34863           Oklahoma City        R276   
9958    Q582715             2974 Holden         P31   
9959    Q582147  Snow Falling on Cedars        P136   
9960    Q458750         Lucille Clifton         P27   

                                               Question  
0                  Which genre of album is harderfaster  
1                     what city was alex golfis born in  
2                   what film is by the writer phil hay  
3                           Where did roger marquis die  
4             what was the cause of death of yves

In [ ]:
test_answerable_dataset['entity_label'] = test_answerable_dataset['entity_label'].apply(lambda x:Preprocessing(x))
new_columns = ["entity_id","entity_label","relation_id","Question"]
test_answerable_dataset = test_answerable_dataset.reindex(columns=new_columns)
print(test_answerable_dataset)

     entity_id       entity_label relation_id  \
0     Q7358590      Roger Marquis         P20   
1      Q154335         Yves Klein        P509   
2     Q2747238       Carlos Gomez        P413   
3       Q62498  Engelbert Zaschka         P21   
4      Q182485      Pee Wee Reese        P413   
...        ...                ...         ...   
5617     Q1492          Barcelona         R19   
5618  Q2870425           Jun Lana       R1431   
5619   Q445899    Gunnar Johansen         P19   
5620   Q582715        2974 Holden         P31   
5621   Q458750    Lucille Clifton         P27   

                                               Question  
0                           Where did roger marquis die  
1             what was the cause of death of yves klein  
2                  What position does carlos gomez play  
3                  how does engelbert zaschka identify   
4     what position does pee wee reese play in baseball  
...                                                 ...  
5617 

**Creation of the corresponding .csv files for the SimpleQuestions test dataset both for the case if this dataset includes answerable and non-answerable over wikidata questions and for the case if this dataset includes only answerable questions over wikidata respectively through pandas method to_csv().**

In [ ]:
import csv
test_csv_name = "drive/MyDrive/test_dataset.csv"
test_dataset.to_csv(test_csv_name, index=False)
test_answerable_csv_name = "drive/MyDrive/test_answerable_dataset.csv"
test_answerable_dataset.to_csv(test_answerable_csv_name, index=False)
print(test_dataset)
print(test_answerable_dataset)

      entity_id            entity_label relation_id  \
0      Q5487302          Harder  Faster        P136   
1     Q16330302             Alex Golfis         P19   
2     Q16225521                Phil Hay         R58   
3      Q7358590           Roger Marquis         P20   
4       Q154335              Yves Klein        P509   
...         ...                     ...         ...   
9956   Q1447249          Doctor Faustus        P170   
9957     Q34863           Oklahoma City        R276   
9958    Q582715             2974 Holden         P31   
9959    Q582147  Snow Falling on Cedars        P136   
9960    Q458750         Lucille Clifton         P27   

                                               Question  
0                  Which genre of album is harderfaster  
1                     what city was alex golfis born in  
2                   what film is by the writer phil hay  
3                           Where did roger marquis die  
4             what was the cause of death of yves